In [7]:

import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np 
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from tensorflow.keras.callbacks import EarlyStopping
from keras.models import load_model
        
df_spam = pd.read_csv("C:\deeplearning\spam_data\SPAM text message 20170820 - Data.csv", header=None, sep=",", names = ["Type", "Message"], encoding='utf-8')
type_train, message_train = df_spam["Type"], df_spam["Message"]

tokenizer=Tokenizer(num_words=df_spam.shape[0])
tokenizer.fit_on_texts(message_train)

message_token=tokenizer.texts_to_sequences(message_train)
token_data=pad_sequences(message_token, maxlen=1500)
encode=LabelEncoder()

type1=encode.fit_transform(df_spam["Type"])
type_data=np_utils.to_categorical(type1)

msg_train=token_data[:int(0.85*len(message_train))]
msg_val=token_data[int(0.85*len(message_train)):]
ty_train=type_data[:int(0.85*len(type_train))]
ty_val=type_data[int(0.85*len(type_train)):]

model=Sequential()
model.add(Embedding(len(message_train), 64, input_length=1500))
model.add(LSTM(50))
model.add(Dropout(0.5))
model.add(Dense(2, activation="softmax"))

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
early_stop=EarlyStopping(monitor="val_loss")
model.fit(msg_train, ty_train, validation_data=(msg_val, ty_val), epochs=10, callbacks=[early_stop])

model.save("spam_model.tf")
model = load_model('spam_model.tf')

def get_key(value):
    dictionary={'ham':0,'spam':1}
    for key,val in dictionary.items():
          if (val==value):
            return key

        
def predict(sentence):
  sentence_lst=[]
  sentence_lst.append(sentence)
  sentence_seq=tokenizer.texts_to_sequences(sentence_lst)
  sentence_padded=pad_sequences(sentence_seq,maxlen=100,padding='post')
  ans=get_key(model.predict_classes(sentence_padded))
  print("The emotion predicted is",ans)
    
predict(str(input('Enter a sentence : ')))

























    
    
    
    
    
    
    
    
    
    

Enter a sentence :  Hello, you just won $50,000!


The emotion predicted is ham
